In [11]:
import pandas as pd
import nltk, re, string, collections
import unicodedata
from nltk.corpus import stopwords
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [23]:
#Intent Classifier
intents = pd.read_csv('intents_classifier.csv',encoding='latin1')
intents

,Text,Intent
0,yes,1
1,yeah,1
2,Yes,1
3,Yeah,1
4,yep,1
...,...,...
109,not as of now,0
110,definitely not,0
111,I think not,0
112,I think no,0


In [24]:
intents['Text'] = intents.apply(lambda x: x['Text'].lower(),axis=1)
intents["Text"] = intents['Text'].str.replace('[^\w\s]','')


In [25]:
X_train, X_test, y_train, y_test = train_test_split(intents['Text'], 
                                                    intents['Intent'], 
                                                    random_state=1)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# Fit the CountVectorizer to the training data
vect = CountVectorizer().fit(X_train)

In [27]:
X_train_vectorized = vect.transform(X_train)

X_train_vectorized

<85x70 sparse matrix of type '<class 'numpy.int64'>'
	with 202 stored elements in Compressed Sparse Row format>

In [28]:
from sklearn.linear_model import LogisticRegression

# Train the model
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

LogisticRegression()

In [29]:
from sklearn.metrics import roc_auc_score

# Predict the transformed test documents
predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.7


In [34]:
predictions1 = model.predict(vect.transform(['I think I have it']))
predictions1

array([1], dtype=int64)

In [40]:
from sklearn.naive_bayes import MultinomialNB 
vectorizer = CountVectorizer() 
X_train_transformed = vectorizer.fit_transform(X_train) 
X_test_transformed = vectorizer.transform(X_test)

clf = MultinomialNB(alpha=0.1) 
clf.fit(X_train_transformed, y_train)

y_predicted = clf.predict(X_test_transformed)
print('AUC: ', roc_auc_score(y_test, y_predicted))

AUC:  0.85


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

9

In [37]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.65


In [39]:
vectorizer = TfidfVectorizer(min_df=3)
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

clf = MultinomialNB(alpha=0.1)
clf.fit(X_train_transformed, y_train)

# y_predicted_prob = clf.predict_proba(X_test_transformed)[:, 1]
y_predicted = clf.predict(X_test_transformed)
print('AUC: ', roc_auc_score(y_test, y_predicted))

AUC:  0.7


In [49]:
y_predicted = clf.predict(vectorizer.transform(['I dont know']))
print(y_predicted)                          

[0]


In [50]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)	

now = 2021-05-20 17:48:03.376279
date and time = 20/05/2021 17:48:03
